**Required imports**

In [ ]:
import json
import jmespath
from dotenv import load_dotenv
from groq import Groq

**Load the JSON File**  
Lod the JSON file which has survey data arranged in multi level nesting  

In [ ]:
# Load JSON data from file
with open("survey_data.json", "r") as f:
    data = json.load(f)

>Key nesting is used as part of query.  
>Filter done at innesr most level (records)  


In [ ]:
# Search for volunteers in Jordan with education "High School"
query = "Jordan.volunteers[?education=='High School']"
results = jmespath.search(query, data)

# Print the filtered records
for person in results:
    print(person)

>Loop through at a level, to retain value of that field

In [ ]:
# Extract names of urban residents per country
urban_names_per_country = {}

# Loop through the outer dimension
for country, info in data.items():
    #query to get names of volunteers living in urban areas
    query = "volunteers[?location_type=='Urban'].name"
    names = jmespath.search(query, info)
    urban_names_per_country[country] = names

urban_names_per_country


In [ ]:
# Dictionary to hold results
phd_demographics_per_country = {}

# Apply JMESPath filter for each country
for country, info in data.items():
    result = jmespath.search("volunteers[?education=='PhD'][].{age: age, sex: sex}", info)
    if result:  # Only store countries with PhD individuals
        phd_demographics_per_country[country+' - PhD'] = result

phd_demographics_per_country

**Retrived Data to LLM**  
The data retrieved from JSON is provided to LLM as context  
This provides the relevant data to respond to the prompt  


In [ ]:
load_dotenv()
client = Groq()
# client = Groq(api_key="Your Key")

>Specific instuction for Response

In [ ]:
R_Instr = "Using the context given, provide response to the user question or statement.\
            Context is provided in JSON format.\
            Provide a comprehensive response"

>The data is being retireved from JSON using list comprehension  


In [ ]:
# User prompt
Prompt = "Who is more educated, in each country? Ladies or Gents?"

# Filter out only relevant fields
filtered_data = {
    country: [
        {"sex": v["sex"], "education": v["education"]}
        for v in info["volunteers"]
    ]
    for country, info in data.items()
}

Context = json.dumps (filtered_data)

# Invoke LLM with prompt and context
messages=[
    {
        "role": "system",
        "content": R_Instr
    },

    {
        "role": "user",
        "content":"Context : \n"+ Context
    },

    {
        "role": "user",
        "content": "Query : \n" + Prompt
    }
]
completion = client.chat.completions.create(
    messages=messages,
    model="openai/gpt-oss-120b",
)

print (completion.choices[0].message.content)